In [1]:
from bs4 import BeautifulSoup
import requests
from time import sleep

In [4]:
def scrape_from_page(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find("header", {"class" : "article-title"}).find("h1").get_text()
    content = "".join([elt.get_text() for elt in soup.find("div", {"id" : "article_content"}).find_all("p")])
    summary = soup.find("h2", {"class" : "subline"}).get_text()
    date = soup.time.attrs['datetime'][:10]

    return [title, date, url, summary, content]

In [5]:
def get_topic_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    urls = list(set([elt.find("a")["href"] for elt in soup.find_all("article")]))
    return list(filter(lambda k: 'https' in k, urls))

In [6]:
def scrape(page_no, topic):
    URL = "https://www.ibtimes.co.in/" + topic + "page/" + str(page_no)
    url_list = get_topic_pages(requests.get(URL))
    sleep(1)

    data = []

    for url in url_list:
        data.append(scrape_from_page(url))
        sleep(1)

    return data

In [18]:
def scrape_loop(start_pg, end_pg, topic):
    data = []
    for pg in range(start_pg, end_pg):
        data += scrape(pg, topic)
        
    return data

In [19]:
dt = scrape_loop(1, 20, "sports")

# Adding to database

In [8]:
import sqlite3

In [9]:
database = "news_db.sqlite"
table_name = "articles"

In [10]:
conn = sqlite3.connect(database)

In [27]:
create = f'CREATE TABLE {table_name} (title VARCHAR, date DATE, url VARCHAR, summary VARCHAR, content VARCHAR)'
delete = f'DROP TABLE {table_name}'

def run(conn, query):
    conn.cursor().execute(query)
    conn.commit()
    
def table_exists(conn, table_name):
    return conn.cursor().execute(
  f"""SELECT name FROM sqlite_master WHERE type='table'
  AND name='{table_name}'; """).fetchall() != []

def show_table(conn, table_name):
    return conn.execute(f"SELECT * FROM {table_name}").fetchall()
    
def insert_data(conn, table_name, data):
    insert = f"""
    INSERT INTO {table_name}
    (title, date, url, summary, content)
    VALUES (?, ?, ?, ?, ?)
    """
    conn.cursor().executemany(insert, data)

In [15]:
# run(conn, delete)

In [16]:
if not (table_exists(conn, "articles")): run(conn, create)

In [24]:
insert_data(conn, "articles", dt)

In [ ]:
show_table(conn, "articles")

In [94]:
conn.close()